In [8]:
import pandas as pd 
import os
from itertools import chain
from collections import defaultdict
from konlpy.utils import pprint 
import numpy as np

In [9]:
# file_name에서 관광지명을 가져온다.
def get_site_name(file_name):
    remove_space = file_name.replace(' ', '')
    output = remove_space.split('.')[1].split('_')[0]
    return output

# file_name에서 '모수부족' file_name과 topkeyword 문구가 없는 file_name을 제거한다.
def get_file_name(file_names):
    for file_name in file_names:
        if '모수부족' in file_name:
            file_names.remove(file_name)
        elif 'topkeyword' not in file_name:
            file_names.remove(file_name)
                        
    return file_names

In [23]:
'''
path: data가 저장되어 있는 경로
'''

path = './data'

file_names = get_file_name(os.listdir(path))

first_dict = dict()
smaple_output = defaultdict(list)


for file_name in file_names:

    # 관광지명 가져오기
    site_name = get_site_name(file_name)

    # 관광지명에 괄호가 존재할 경우 괄호 앞에 부분을 관광지명으로 정의
    if '(' in site_name:
        site_name = site_name.split('(')[0]

    if '[' in site_name:
        site_name = site_name.split('[')[0]

    if 'csv' in file_name:
        df = pd.read_csv(f'{path}/{file_name}', encoding='cp949')

    # 데이터분할 파일을 xlsx 형태
    if 'xlsx' in file_name:
        df = pd.read_excel(f'{path}/{file_name}', sheet_name=0, skiprows = 3)

    # 연관어 가져오기 
    keyword_freq = df[['key', 'freq']]
    
    # 연관어에서 관광지 명 삭제
    keyword_freq_filter = keyword_freq[keyword_freq.key != site_name]
    keyword_freq_filter = keyword_freq_filter[keyword_freq_filter['key'].notnull()]

    get_dict = keyword_freq_filter.set_index('key')['freq'].to_dict()
    
    for k, v in chain(first_dict.items(), get_dict.items()):
        smaple_output[k].append(v)
        
    print(file_name)

01. 강릉커피거리_topkeyword.csv


In [24]:
# group_by_sum = dict()
# a = smaple_output.items()

# for k, v in a:
#     group_by_sum[k] = sum(v)

In [19]:
# https://mr-doosun.tistory.com/22
# https://github.com/koshort/pyeunjeon 
from konlpy.tag import  Kkma, Okt, Hannanum, Komoran
from eunjeon import Mecab

kkma = Kkma() # 서울대 개발
okt = Okt() # 이전 트위터
han = Hannanum() # 카이스트 개발
komoran = Komoran() # Shineware 개발
mecab = Mecab() # ! pip install eunjeon

In [31]:
a = ['추운지도', '싱그러움이', '철썩철썩', '보내기', '여기저기', '서울대입구요가', '여행이었으']

for key in a:
    print('원본:'+key)
    print('Hannanum'+' -> '+str(han.nouns(key)))
    print('komoran'+' -> '+str(komoran.nouns(key)))
    print('okt'+' -> '+str(okt.nouns(key)))    
    print('mecab'+' -> '+str(mecab.nouns(key)))    
    print('='*20)            

원본:추운지도
Hannanum -> []
komoran -> []
okt -> []
mecab -> ['지도']
원본:싱그러움이
Hannanum -> []
komoran -> []
okt -> ['움']
mecab -> []
원본:철썩철썩
Hannanum -> []
komoran -> []
okt -> ['철썩', '철썩']
mecab -> []
원본:보내기
Hannanum -> []
komoran -> []
okt -> []
mecab -> []
원본:여기저기
Hannanum -> []
komoran -> ['여기저기']
okt -> ['여기저기']
mecab -> ['여기저기']
원본:서울대입구요가
Hannanum -> ['서울대입구요']
komoran -> ['서울대', '입구', '요가']
okt -> ['서울대', '입구', '요가']
mecab -> ['서울대', '입구', '요가']
원본:여행이었으
Hannanum -> ['여행이었으']
komoran -> ['여행', '으']
okt -> ['여행']
mecab -> ['여행']


In [26]:
a = keyword_freq_filter['key'].tolist()

li = list()
for key in a:
    temp_list = list()
    temp_list.append(key)
    try:
        xx = han.nouns(key)
    except: 
        print(key)
    if len(xx) == 0:
        temp_list.append(' ')
    else:
        temp_list.append(xx[0])

    li.append(temp_list)

In [27]:
pd.DataFrame(li).to_csv('dddd.csv', index=False, encoding='utf-8-sig')
